In [1]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [14]:
sc.textFile('cikm2020/canonical-link-graph/feature-set-precision-experiments/cw09-1-gramms-raw-data.jsonl').take(2)

['{"candidate":{"featureName":"1-gramms","firstId":"clueweb09-enwp01-14-02419","secondId":"clueweb09-enwp02-06-18934"},"leftDoc":{"id":"clueweb09-enwp01-14-02419","content":"Pope Benedict XII - Wikipedia, the free encyclopedia Pope Benedict XII From Wikipedia, the free encyclopedia \xa0\xa0(Redirected from Jacques fornier) Jump to: navigation, search This article does not cite any references or sources. Please help improve this article by adding citations to reliable sources. Unverifiable material may be challenged and removed. (March 2009) Benedict XII Papacy began December 20, 1334 Papacy ended April 25, 1342 Predecessor John XXII Successor Clement VI Birth name Jacques Fournier Born ca. 1280s Saverdun, France Died April 25, 1342 Avignon, France Other popes named Benedict Papal styles of Pope Benedict XII Reference style His Holiness Spoken style Your Holiness Religious style Holy Father Posthumous style None Pope Benedict XII (died April 24, 1342), born Jacques Fournier, was Pope fr

In [2]:
def map_to_precision_recall_counter(src, threshold):
    import json
    parsed = json.loads(src)
    truePositive = float(parsed['s3Score']) >= threshold
    
    return (parsed['candidate']['featureName'] , {
        'truePositiveCount': 1 if truePositive else 0,
        'falsePositiveCount': 0 if truePositive else 1
    })

def comb(i, j):
    return {
        'truePositiveCount': i['truePositiveCount']+j['truePositiveCount'],
        'falsePositiveCount': i['falsePositiveCount']+j['falsePositiveCount']
    }

def extract_raw_precision_experiment_results(sc, collection, threshold):
    zeroValue = {'truePositiveCount': 0, 'falsePositiveCount': 0}
    return sc.textFile('cikm2020/canonical-link-graph/feature-set-precision-experiments/' + collection + '-*-gramms-raw-data.jsonl')\
        .map(lambda i: map_to_precision_recall_counter(i, threshold))\
        .aggregateByKey(zeroValue, comb, comb)\
        .collect()


In [3]:
threshold = 0.8
raw_data_cw_09 = extract_raw_precision_experiment_results(sc, 'cw09', threshold)
raw_data_cw_12 = extract_raw_precision_experiment_results(sc, 'cw12', threshold)
#raw_data_cc_15 = extract_raw_precision_experiment_results(sc, 'cc-2015-11', threshold)
raw_data_cc_17 = extract_raw_precision_experiment_results(sc, 'cc-2017-04', threshold)

In [26]:
import json

json.dumps(raw_data_cw_09)

'[["1-8-gramms", {"truePositiveCount": 40566, "falsePositiveCount": 9434}], ["5-8-gramms", {"truePositiveCount": 49972, "falsePositiveCount": 28}], ["3-5-gramms", {"truePositiveCount": 49564, "falsePositiveCount": 436}], ["3-8-gramms", {"truePositiveCount": 49599, "falsePositiveCount": 401}], ["1-gramms", {"truePositiveCount": 30162, "falsePositiveCount": 19838}], ["1-3-gramms", {"truePositiveCount": 40816, "falsePositiveCount": 9184}], ["8-gramms", {"truePositiveCount": 49944, "falsePositiveCount": 56}], ["5-gramms", {"truePositiveCount": 49781, "falsePositiveCount": 219}], ["3-gramms", {"truePositiveCount": 48766, "falsePositiveCount": 1234}], ["1-5-gramms", {"truePositiveCount": 34428, "falsePositiveCount": 15572}]]'

In [38]:
def raw_precision_data_to_df(raw_data, label):
    import pandas as pd

    ret = pd.DataFrame([[i[0], i[1]['truePositiveCount'], i[1]['falsePositiveCount'], label] for i in raw_data], columns=['feature', 'truePositiveCount', 'falsePositiveCount', 'label'])
    ret['examples'] = ret['truePositiveCount'] + ret['falsePositiveCount']
    ret['precision'] = ret['truePositiveCount']/ret['examples']

    return ret

In [8]:
raw_precision_data_to_df(raw_data_cw_09, 'cw09').sort_values('precision', ascending=False)

,feature,truePositiveCount,falsePositiveCount,label,examples,precision
1,5-8-gramms,49972,28,cw09,50000,0.99944
6,8-gramms,49944,56,cw09,50000,0.99888
7,5-gramms,49781,219,cw09,50000,0.99562
3,3-8-gramms,49599,401,cw09,50000,0.99198
2,3-5-gramms,49564,436,cw09,50000,0.99128
8,3-gramms,48766,1234,cw09,50000,0.97532
5,1-3-gramms,40816,9184,cw09,50000,0.81632
0,1-8-gramms,40566,9434,cw09,50000,0.81132
9,1-5-gramms,34428,15572,cw09,50000,0.68856
4,1-gramms,30162,19838,cw09,50000,0.60324


In [9]:
raw_precision_data_to_df(raw_data_cw_12, 'cw12').sort_values('precision', ascending=False)

,feature,truePositiveCount,falsePositiveCount,label,examples,precision
2,3-5-gramms,48613,1387,cw12,50000,0.97226
1,5-8-gramms,48572,1428,cw12,50000,0.97144
3,3-8-gramms,48400,1600,cw12,50000,0.96800
7,5-gramms,45541,4459,cw12,50000,0.91082
8,3-gramms,44748,5252,cw12,50000,0.89496
6,8-gramms,44424,5576,cw12,50000,0.88848
5,1-3-gramms,22479,27521,cw12,50000,0.44958
0,1-8-gramms,22022,27978,cw12,50000,0.44044
9,1-5-gramms,21800,28200,cw12,50000,0.43600
4,1-gramms,17439,32561,cw12,50000,0.34878


In [10]:
raw_precision_data_to_df(raw_data_cc_17, 'cc-2017-04').sort_values('precision', ascending=False)

,feature,truePositiveCount,falsePositiveCount,label,examples,precision
2,3-5-gramms,47322,2678,cc-2017-04,50000,0.94644
3,3-8-gramms,46959,3041,cc-2017-04,50000,0.93918
8,3-gramms,42997,7003,cc-2017-04,50000,0.85994
7,5-gramms,41788,8212,cc-2017-04,50000,0.83576
1,5-8-gramms,41396,8604,cc-2017-04,50000,0.82792
6,8-gramms,33268,16732,cc-2017-04,50000,0.66536
9,1-5-gramms,33239,16761,cc-2017-04,50000,0.66478
5,1-3-gramms,32844,17156,cc-2017-04,50000,0.65688
0,1-8-gramms,31490,18510,cc-2017-04,50000,0.62980
4,1-gramms,29131,20869,cc-2017-04,50000,0.58262


In [51]:
def line_table_feature_sets_precision(feature, df):
    def c(label):
        return '{:.3f}'.format(df[(df['feature'] == feature) & (df['label'] == label)].values[0][5])
    return feature + ' & ' + c('cw09') +' & ' + c('cw12') +' & ??? & ' + c('cc17') +'\\\\';

def table_feature_sets_precision(df):
    features = ['1-gramms', '3-gramms', '5-gramms', '8-gramms']
    combined_features = ['1-3-gramms', '1-5-gramms', '1-8-gramms', '3-5-gramms', '3-8-gramms', '5-8-gramms']
    
    content = '\n\n'.join(line_table_feature_sets_precision(i, df) for i in features)
    content += '\n\n\\midrule\n\n'
    content += '\n\n'.join(line_table_feature_sets_precision(i, df) for i in combined_features)
    
    return """
\\begin{table*}
\\centering
\\small
\\setlength{\\tabcolsep}{3pt}%
\\caption{TBD}
\\label{table-feature-sets-precision}
\\begin{tabular}{@{}l@{\\hspace{2\\tabcolsep}}c@{\\hspace{3\\tabcolsep}}c@{\\hspace{3\\tabcolsep}}c@{\\hspace{3\\tabcolsep}}c@{}}

\\toprule
\\bfseries Feature & \\multicolumn{4}{c@{}}{\\bfseries Precision}\\\\

\\cmidrule(r){2-5}
& CW09 & CW12 & CC15 & CC 17\\\\
\\midrule

""" + content + """

\\bottomrule

\\end{tabular}
\\end{table*} 
 
"""

df = raw_precision_data_to_df(raw_data_cw_09, 'cw09')
df = df.append(raw_precision_data_to_df(raw_data_cw_12, 'cw12'))
df = df.append(raw_precision_data_to_df(raw_data_cc_17, 'cc17'))

print(table_feature_sets_precision(df))


\begin{table*}
\centering
\small
\setlength{\tabcolsep}{3pt}%
\caption{TBD}
\label{table-feature-sets-precision}
\begin{tabular}{@{}l@{\hspace{2\tabcolsep}}c@{\hspace{3\tabcolsep}}c@{\hspace{3\tabcolsep}}c@{\hspace{3\tabcolsep}}c@{}}

\toprule
\bfseries Feature & \multicolumn{4}{c@{}}{\bfseries Precision}\\

\cmidrule(r){2-5}
& CW09 & CW12 & CC15 & CC 17\\
\midrule

1-gramms & 0.603 & 0.349 & ??? & 0.583\\

3-gramms & 0.975 & 0.895 & ??? & 0.860\\

5-gramms & 0.996 & 0.911 & ??? & 0.836\\

8-gramms & 0.999 & 0.888 & ??? & 0.665\\

\midrule

1-3-gramms & 0.816 & 0.450 & ??? & 0.657\\

1-5-gramms & 0.689 & 0.436 & ??? & 0.665\\

1-8-gramms & 0.811 & 0.440 & ??? & 0.630\\

3-5-gramms & 0.991 & 0.972 & ??? & 0.946\\

3-8-gramms & 0.992 & 0.968 & ??? & 0.939\\

5-8-gramms & 0.999 & 0.971 & ??? & 0.828\\

\bottomrule

\end{tabular}
\end{table*} 
 



0.81132

In [26]:
raw_precision_data_to_df(raw_data_cw_09, 'cw09').sort_values('precision', ascending=False)

TypeError: tuple indices must be integers or slices, not str

In [33]:
import json
json.dumps(raw_data_cw_09)

'[["1-8-gramms", {"truePositiveCount": 40566, "falsePositiveCount": 9434}], ["5-8-gramms", {"truePositiveCount": 49972, "falsePositiveCount": 28}], ["3-5-gramms", {"truePositiveCount": 49564, "falsePositiveCount": 436}], ["3-8-gramms", {"truePositiveCount": 49599, "falsePositiveCount": 401}], ["1-gramms", {"truePositiveCount": 30162, "falsePositiveCount": 19838}], ["1-3-gramms", {"truePositiveCount": 40816, "falsePositiveCount": 9184}], ["8-gramms", {"truePositiveCount": 49944, "falsePositiveCount": 56}], ["5-gramms", {"truePositiveCount": 49781, "falsePositiveCount": 219}], ["3-gramms", {"truePositiveCount": 48766, "falsePositiveCount": 1234}], ["1-5-gramms", {"truePositiveCount": 34428, "falsePositiveCount": 15572}], [["1-8-gramms", {"truePositiveCount": 22022, "falsePositiveCount": 27978}], ["5-8-gramms", {"truePositiveCount": 48572, "falsePositiveCount": 1428}], ["3-5-gramms", {"truePositiveCount": 48613, "falsePositiveCount": 1387}], ["3-8-gramms", {"truePositiveCount": 48400, "fa

In [37]:
raw_data_cw_09 = json.loads('[["1-8-gramms", {"truePositiveCount": 40566, "falsePositiveCount": 9434}], ["5-8-gramms", {"truePositiveCount": 49972, "falsePositiveCount": 28}], ["3-5-gramms", {"truePositiveCount": 49564, "falsePositiveCount": 436}], ["3-8-gramms", {"truePositiveCount": 49599, "falsePositiveCount": 401}], ["1-gramms", {"truePositiveCount": 30162, "falsePositiveCount": 19838}], ["1-3-gramms", {"truePositiveCount": 40816, "falsePositiveCount": 9184}], ["8-gramms", {"truePositiveCount": 49944, "falsePositiveCount": 56}], ["5-gramms", {"truePositiveCount": 49781, "falsePositiveCount": 219}], ["3-gramms", {"truePositiveCount": 48766, "falsePositiveCount": 1234}], ["1-5-gramms", {"truePositiveCount": 34428, "falsePositiveCount": 15572}]]')